# Riesgo de Mercado

## Cálculo de Volatilidad

### Volatilidad de un activo financiero

In [11]:
import numpy as np
import pandas as pd

# Generar datos de ejemplo
np.random.seed(42)
days = 252 * 3  # 3 años de datos diarios
returns = np.random.normal(0, 0.01, days)  # Rendimientos
returns_series = pd.Series(returns)

# Calcular volatilidad anualizada
volatility = returns_series.std() * np.sqrt(252)
print(f'Volatilidad anualizada: {volatility:.2%}')

Volatilidad anualizada: 15.69%


### Volatilidad de una cartera de activos

In [12]:
import numpy as np
import pandas as pd

# Simular rendimientos de 4 activos
num_assets = 4
returns_matrix = np.random.normal(0, 0.01, (days, num_assets))
returns_df = pd.DataFrame(returns_matrix, columns=[f'Asset_{i+1}' for i in range(num_assets)])

# Pesos de la cartera
weights = np.array([0.25, 0.25, 0.25, 0.25])
# Calcular la matriz de covarianza
cov_matrix = returns_df.cov() 

# Calcular la volatilidad de la cartera
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix.values, weights)))
portfolio_volatility_annualized = portfolio_volatility * np.sqrt(252)
print(f'Volatilidad anual de la cartera: {portfolio_volatility_annualized:.2%}')

Volatilidad anual de la cartera: 7.78%


### Ajuste de volatilidad histórica con EWMA

#### EWMA de un activo financiero

In [13]:
import numpy as np
import pandas as pd

# Función para calcular la volatilidad con EWMA
def ewma_volatility(returns:pd.DataFrame, lambda_:float=0.94) -> pd.Series:
    ewma_var = returns.ewm(alpha=1 - lambda_).var()
    ewma_vol = np.sqrt(ewma_var)
    return ewma_vol 

# Generar datos de ejemplo
np.random.seed(42)
days = 252 * 3  # 3 años de datos diarios
returns = np.random.normal(0, 0.01, days)  # Rendimientos
returns_series = pd.Series(returns)

# Calcular volatilidad con EWMA
ewma_vol = ewma_volatility(returns_series)
ewma_vol_annualized = ewma_vol * np.sqrt(252)
print(f'Volatilidad anualizada con EWMA: {list(ewma_vol_annualized)[-1]:.2%}')

Volatilidad anualizada con EWMA: 18.21%


####  EWMA de una cartera de activos

In [14]:
import numpy as np
import pandas as pd   
# Función para calcular la matriz de covarianza con EWMA
def ewma_covariance(returns:pd.DataFrame, lambda_:float=0.94) -> pd.DataFrame:
    ewma_cov = returns.ewm(alpha=1 - lambda_).cov()
    return ewma_cov   
# Calcular la matriz de covarianza con EWMA
ewma_cov_matrix = ewma_covariance(returns_df)
# Calcular la volatilidad de la cartera con la matriz EWMA
# Extract the last covariance matrix from the MultiIndex DataFrame
last_cov_matrix = ewma_cov_matrix.xs(ewma_cov_matrix.index.get_level_values(0)[-1], level=0)
portfolio_ewma_volatility = np.sqrt(np.dot(weights.T, np.dot(last_cov_matrix.values, weights)))
portfolio_ewma_volatility_annualized = portfolio_ewma_volatility * np.sqrt(252)
print(f'Volatilidad anual de la cartera con EWMA: {portfolio_ewma_volatility_annualized:.2%}')

Volatilidad anual de la cartera con EWMA: 9.10%


## Cálculo de VaR

### Método Paramétrico

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Simular rendimientos de un activo financiero
np.random.seed(42)
days = 252 * 3  # 3 años de datos diarios
returns = np.random.normal(0, 0.01, days)  # Rendimientos
returns_series = pd.Series(returns)

# Parámetros del VaR
confidence_level = 0.95
mean_return = returns_series.mean()
std_dev = returns_series.std()

# Calcular el VaR paramétrico
z_score = norm.ppf(1 - confidence_level)
var_parametric = -(mean_return + z_score * std_dev)
print(f'VaR paramétrico al {confidence_level*100}%: {var_parametric:.2%}')


VaR paramétrico al 95.0%: 1.64%


In [16]:
import numpy as np
import pandas as pd

# Simular rendimientos de 4 activos
num_assets = 4
returns_matrix = np.random.normal(0, 0.01, (days, num_assets))
returns_df = pd.DataFrame(returns_matrix, columns=[f'Asset_{i+1}' for i in range(num_assets)])  

# Pesos de la cartera
weights = np.array([0.25, 0.25, 0.25, 0.25])

# Calcular la matriz de covarianza
cov_matrix = returns_df.cov()

# Calcular la volatilidad de la cartera
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix.values, weights)))
portfolio_volatility_annualized = portfolio_volatility * np.sqrt(252)   

# Parámetros del VaR
confidence_level = 0.95
mean_return = returns_df.dot(weights).mean()
std_dev = portfolio_volatility_annualized
# Calcular el VaR paramétrico de la cartera
z_score = norm.ppf(1 - confidence_level)
var_parametric_portfolio = -(mean_return + z_score * std_dev)
print(f'VaR paramétrico de la cartera al {confidence_level*100}%: {var_parametric_portfolio:.2%}')

VaR paramétrico de la cartera al 95.0%: 12.76%


### Método Histórico

In [17]:
import numpy as np
import pandas as pd

# Simular rendimientos de un activo financiero
np.random.seed(42)
days = 252 * 3  # 3 años de datos diarios
returns = np.random.normal(0, 0.01, days)  # Rendimientos
returns_series = pd.Series(returns)

# Parámetros del VaR
confidence_level = 0.95

# Calcular el VaR histórico
var_historical = -returns_series.quantile(1 - confidence_level)
print(f'VaR histórico al {confidence_level*100}%: {var_historical:.2%}')

VaR histórico al 95.0%: 1.61%


In [18]:
import numpy as np
import pandas as pd
# Simular rendimientos de 4 activos
num_assets = 4
returns_matrix = np.random.normal(0, 0.01, (days, num_assets))
returns_df = pd.DataFrame(returns_matrix, columns=[f'Asset_{i+1}' for i in range(num_assets)])  
# Pesos de la cartera
weights = np.array([0.25, 0.25, 0.25, 0.25])
# Calcular los rendimientos de la cartera
portfolio_returns = returns_df.dot(weights)
# Parámetros del VaR
confidence_level = 0.95
# Calcular el VaR histórico de la cartera
var_historical_portfolio = -portfolio_returns.quantile(1 - confidence_level)
print(f'VaR histórico de la cartera al {confidence_level*100}%: {var_historical_portfolio:.2%}')

VaR histórico de la cartera al 95.0%: 0.76%


### Método Simulación de Monte Carlo

In [19]:
import numpy as np
import pandas as pd

# Simular rendimientos de un activo financiero
np.random.seed(42)
days = 252 * 3  # 3 años de datos diarios
returns = np.random.normal(0, 0.01, days)  # Rendimientos
returns_series = pd.Series(returns) 

# Parámetros del activo
S0 = 100  # Precio inicial
# Retorno anualizado
mu = returns_series.mean() * 252  
# Volatilidad anualizada   
sigma = returns_series.std() * np.sqrt(252)  

# Simulación de Monte Carlo
num_simulations = 10000
time_horizon = 1/252  # 1 día
simulated_prices = []   
for _ in range(num_simulations):
    Z = np.random.normal()
    S1 = S0 * np.exp((mu - 0.5 * sigma**2) * time_horizon + sigma * np.sqrt(time_horizon) * Z)
    simulated_prices.append(S1)
simulated_prices = np.array(simulated_prices)

# Calcular los rendimientos simulados
simulated_returns = (simulated_prices - S0) / S0

# Parámetros del VaR
confidence_level = 0.95

# Calcular el VaR mediante simulación de Monte Carlo
var_monte_carlo = -np.percentile(simulated_returns, (1 - confidence_level) * 100)
print(f'VaR por Simulación de Monte Carlo al {confidence_level*100}%: {var_monte_carlo:.2%}')


VaR por Simulación de Monte Carlo al 95.0%: 1.65%


In [20]:
import numpy as np
import pandas as pd

# Simular rendimientos de 4 activos
num_assets = 4
returns_matrix = np.random.normal(0, 0.01, (days, num_assets))
returns_df = pd.DataFrame(returns_matrix, columns=[f'Asset_{i+1}' for i in range(num_assets)])

# Pesos de la cartera
weights = np.array([0.25, 0.25, 0.25, 0.25]) 

# Parámetros de los activos
S0 = np.array([100, 150, 200, 250])  
mu = returns_df.mean() * 252  
sigma = returns_df.std() * np.sqrt(252)

# Matriz de correlación y descomposición de Cholesky
correlation_matrix = returns_df.corr()
L = np.linalg.cholesky(correlation_matrix)

# Simulación de Monte Carlo con Cholesky
num_simulations = 10000
time_horizon = 1/252  # 1 día
simulated_portfolio_returns = []   
for _ in range(num_simulations):
    Z_independent = np.random.normal(size=num_assets)
    # Correlacionar los shocks
    Z_correlated = L @ Z_independent  
    S1 = S0 * np.exp((mu - 0.5 * sigma**2) * time_horizon + sigma * np.sqrt(time_horizon) * Z_correlated)   
    portfolio_return = np.dot(weights, (S1 - S0) / S0)
    simulated_portfolio_returns.append(portfolio_return)
simulated_portfolio_returns = np.array(simulated_portfolio_returns)     

# Parámetros del VaR
confidence_level = 0.95 

# Calcular el VaR mediante simulación de Monte Carlo para la cartera
var_monte_carlo_portfolio = -np.percentile(simulated_portfolio_returns, (1 - confidence_level) * 100)
print(f'VaR por Simulación de Monte Carlo de la cartera al {confidence_level*100}%: {var_monte_carlo_portfolio:.2%}')

VaR por Simulación de Monte Carlo de la cartera al 95.0%: 0.82%
